# 3. Categorical Variables

## Introduction
1. categorical variables이란?
  - 제한된 개수의 값만 존재하는 변수
  - 예시: 설문조사에서 4개의 응답만 가능한 경우(Never, Rarely, Most days, Every day)
2. 전처리 방법
  - Drop: categorical variable을 dataset에서 제거하여 모델을 만드는 데 사용하지 않는다. categorical variable이 중요한 정보를 가지고 있지 않을 때만 유용한 방법이다.
  - Ordinal Encoding: 각 고유값(unique value)마다 서로 다른 정수를 부여한다. 이때 부여되는 정수들은 순서를 내포하고 있다. 카테고리가 indisputable ranking인 경우에만 사용 가능하다.
    - 예시: Never(0) < Rarely(1) < Most days(2) < Every day(3)
  - One-Hot Encoding: 각 고유값들의 presence(absence)를 나타내는 칼럼을 새로 만든다. nominal variables에 적용한다.
    - nominal variables: 카테고리에 명확한 ranking이 없는 변수
    - 카테고리의 값이 너무 많으면 잘 작동하지 않는다(15개 이하의 값을 갖는 경우에만 사용하는 것이 일반적)


## exercise

In [1]:
# basic settings
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Read the data
X = pd.read_csv('data/housing_prices_train.csv', index_col='Id') 
X_test = pd.read_csv('data/housing_prices_test.csv', index_col='Id')

# Remove rows with missing target, separate target from predictors
X.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X.SalePrice
X.drop(['SalePrice'], axis=1, inplace=True)

# To keep things simple, we'll drop columns with missing values
cols_with_missing = [col for col in X.columns if X[col].isnull().any()] 
X.drop(cols_with_missing, axis=1, inplace=True)
X_test.drop(cols_with_missing, axis=1, inplace=True)

# Break off validation set from training data
X_train, X_valid, y_train, y_valid = train_test_split(X, y,
                                                      train_size=0.8, test_size=0.2,
                                                      random_state=0)


# function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

### 첫 번째 접근: categorical variables 삭제

In [2]:
drop_X_train = X_train.select_dtypes(exclude=['object'])
drop_X_valid = X_valid.select_dtypes(exclude=['object'])

score_dataset(drop_X_train, drop_X_valid, y_train, y_valid)

17837.82570776256

### 두 번째 접근: ordinal encoding
- 유의점: training data에는 나타나지 않지만 validation data에는 나타나는 값이 존재할 수 있다. 이 경우 training data에 fit된 OrdinalEncoder로 validation data를 transform할 때 에러가 발생한다.
  - 해결법: 칼럼 자체를 삭제 / custom ordinal encoder 정의

In [3]:
set(X_train.MSSubClass)

{20, 30, 40, 45, 50, 60, 70, 75, 80, 85, 90, 120, 160, 180, 190}

In [4]:
# 칼럼들을 위에 언급된 유의점의 문제가 발생하는 칼럼과 발생하지 않는 칼럼으로 나누기

# object 타입의 모든 칼럼
obj_cols = list(X_train.select_dtypes(['object']).columns)
# cat_cols = [col for col in X_train.columns if X_train[col].dtype == 'object']

# 각 obj_cols의 값들 중 validation에 존재하는 값들이 모두 train에도 존재하는 칼럼만 추출
good_cols = [col for col in obj_cols if set(X_valid[col]).issubset(set(X_train[col]))]
bad_cols = list(set(obj_cols) - set(good_cols))
print(f'good_cols: {good_cols}')
print(f'bad_cols: {bad_cols}')

good_cols: ['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'BldgType', 'HouseStyle', 'RoofStyle', 'Exterior1st', 'Exterior2nd', 'ExterQual', 'ExterCond', 'Foundation', 'Heating', 'HeatingQC', 'CentralAir', 'KitchenQual', 'PavedDrive', 'SaleType', 'SaleCondition']
bad_cols: ['RoofMatl', 'Condition2', 'Functional']


In [5]:
# bad_cols를 dataset에서 drop
od_X_train = X_train.drop(bad_cols, axis=1)
od_X_valid = X_valid.drop(bad_cols, axis=1)

In [6]:
# ordinal encoding
from sklearn.preprocessing import OrdinalEncoder

ordinal_encoder = OrdinalEncoder()
od_X_train[good_cols] = ordinal_encoder.fit_transform(od_X_train[good_cols])
od_X_valid[good_cols] = ordinal_encoder.transform(od_X_valid[good_cols])

In [7]:
# result
score_dataset(od_X_train, od_X_valid, y_train, y_valid)

17098.01649543379

### 세 번째 접근: one-hot encoding
- 유의점
  1. 각 object 칼럼들이 몇 개의 unique values를 가지고 있는지 먼저 파악해야 한다(=cardinality).
  2. 행이 많은 dataset일 경우, one-hot encoding 진행 시 dataset 크기가 매우 커진다.
- 처리 순서
  1. 인코딩 후 DataFrame 형태로 결과를 받는다.
  2. index를 되돌린다.
  3. 인코딩 전 기존 object 칼럼들을 삭제한다.
  4. numeric 칼럼들과 합쳐준다.
  5. 모델 build 및 테스트

In [8]:
# cardinality 구하기
obj_unique_nums = list(map(lambda col: X_train[col].nunique(), obj_cols))
d = dict(zip(obj_cols, obj_unique_nums))
print(d)

{'MSZoning': 5, 'Street': 2, 'LotShape': 4, 'LandContour': 4, 'Utilities': 2, 'LotConfig': 5, 'LandSlope': 3, 'Neighborhood': 25, 'Condition1': 9, 'Condition2': 6, 'BldgType': 5, 'HouseStyle': 8, 'RoofStyle': 6, 'RoofMatl': 7, 'Exterior1st': 15, 'Exterior2nd': 16, 'ExterQual': 4, 'ExterCond': 5, 'Foundation': 6, 'Heating': 6, 'HeatingQC': 5, 'CentralAir': 2, 'KitchenQual': 4, 'Functional': 6, 'PavedDrive': 3, 'SaleType': 9, 'SaleCondition': 6}


In [9]:
# cardinality가 10 미만인 칼럼 구하기
filter_key = lambda x: x[1] < 10
low_cardinality_cols = list(filter(filter_key, d.items()))
low_cardinality_cols = list(map(lambda x: x[0], low_cardinality_cols))
print(low_cardinality_cols)

['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'ExterQual', 'ExterCond', 'Foundation', 'Heating', 'HeatingQC', 'CentralAir', 'KitchenQual', 'Functional', 'PavedDrive', 'SaleType', 'SaleCondition']


In [26]:
# one-hot encoding
from sklearn.preprocessing import OneHotEncoder

# handle_unknown='ignore': training dataset에서는 나타나지 않은 값이 validation dataset에 나타난 경우 무시 
# sparse=False: 인코딩된 칼럼이 numpy array type을 갖도록
encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
oh_X_train = pd.DataFrame(encoder.fit_transform(X_train[low_cardinality_cols]))
oh_X_valid = pd.DataFrame(encoder.transform(X_valid[low_cardinality_cols]))

# 인덱스 되돌리기
oh_X_train.index = X_train.index
oh_X_valid.index = X_valid.index

# numeric columns
num_X_train = X_train.drop(obj_cols, axis=1)
num_X_valid = X_valid.drop(obj_cols, axis=1)
# num_cols = list(set(X_train.columns) - set(obj_cols))
# num_X_train = X_train[num_cols]
# num_X_valid = X_valid[num_cols]

# categorical, numeric columns를 합쳐서 최종 dataset 만들기
oh_X_train_final = pd.concat([oh_X_train, num_X_train], axis=1)
oh_X_valid_final = pd.concat([oh_X_valid, num_X_valid], axis=1)


In [27]:
# result
score_dataset(oh_X_train_final, oh_X_valid_final, y_train, y_valid)

c:\Users\multicampus\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
c:\Users\multicampus\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


17537.55444063927